# MGnify to Darwin Core export notes
## How are OTUs derived in the MGnify pipeline?

In [25]:
import pandas as pd
from mgnifyextract.analyses import get_analysis
from mgnifyextract.downloads import FastaDownload, MseqDownload, TsvDownload
from mgnifyextract.studies import get_superstudy_studies
from mgnifyextract.util import clean_taxonomy_string

In [26]:
analysis = get_analysis("MGYA00593805")
analysis

<Analysis https://www.ebi.ac.uk/metagenomics/analyses/MGYA00593805>

In [27]:
downloads = analysis.get_downloads()

marker = "SSU"

fasta_files = [download for download in downloads if isinstance(download, FastaDownload) and download.marker == marker]
mseq_files = [download for download in downloads if isinstance(download, MseqDownload) and download.marker == marker]
tsv_files = [download for download in downloads if isinstance(download, TsvDownload) and download.marker == marker]

Let's take a look at the number of rows in the mseq and OTU files.

In [28]:
mseq = mseq_files[0].read()
mseq.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12178 entries, 0 to 12177
Data columns (total 15 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   #query       12178 non-null  object 
 1   dbhit        12178 non-null  object 
 2   bitscore     12178 non-null  int64  
 3   identity     12178 non-null  float64
 4   matches      12178 non-null  int64  
 5   mismatches   12178 non-null  int64  
 6   gaps         12178 non-null  int64  
 7   query_start  12178 non-null  int64  
 8   query_end    12178 non-null  int64  
 9   dbhit_start  12178 non-null  int64  
 10  dbhit_end    12178 non-null  int64  
 11  strand       12178 non-null  object 
 12  Unnamed: 12  0 non-null      float64
 13  SILVA        12178 non-null  object 
 14  Unnamed: 14  0 non-null      float64
dtypes: float64(3), int64(8), object(4)
memory usage: 1.4+ MB


In [29]:
otu = tsv_files[0].read()
otu.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 121 entries, 0 to 120
Data columns (total 4 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   # OTU ID  121 non-null    int64  
 1   SSU_rRNA  121 non-null    float64
 2   taxonomy  121 non-null    object 
 3   taxid     121 non-null    int64  
dtypes: float64(1), int64(2), object(1)
memory usage: 3.9+ KB


The same number of distinct taxonomy values in the mseq file as well as the OTU suggests that the OTUs correspond to all distinct taxonomic assignments regardless of sequence similarity. All reads with assignment Bacteria for example are collapsed into a single OTU.

In [30]:
pd.Series([clean_taxonomy_string(tax) for tax in otu["taxonomy"]]).nunique()

121

In [31]:
pd.Series([clean_taxonomy_string(tax) for tax in mseq["SILVA"]]).nunique()

121

## Alternative approach

Instead of relying on the OTU table as downloaded from MGnify, let's try grouping sequences by SILVA hit.

In [32]:
study = get_superstudy_studies("atlanteco")[0]
study


<Study https://www.ebi.ac.uk/metagenomics/studies/MGYS00005780 >

In [33]:
sample = study.get_samples(max_results=1)[0]
sample

<Sample https://www.ebi.ac.uk/metagenomics/samples/SRS2329696 >

In [34]:
run = sample.get_runs(max_results=1)[0]
run

<Run https://www.ebi.ac.uk/metagenomics/runs/SRR5788044 xx>

In [35]:
analysis = run.get_analyses(max_results=1)[0]
analysis

<Analysis https://www.ebi.ac.uk/metagenomics/analyses/MGYA00463299>

In [36]:
marker = "LSU"
downloads = analysis.get_downloads()
fasta_files = [download for download in downloads if isinstance(download, FastaDownload) and download.marker == marker]
mseq_files = [download for download in downloads if isinstance(download, MseqDownload) and download.marker == marker]
fasta = fasta_files[0].read_pandas()
mseq = mseq_files[0].read()

Merge fasta and mseq tables:

In [37]:
df = fasta.merge(mseq.rename({"#query": "reference"}, axis=1), how="left", on="reference")

First clean taxonomy strings by removing empty ranks:

In [38]:
df["SILVA"] = [clean_taxonomy_string(tax) for tax in df["SILVA"]]

Let's take a look at the number of distinct sequences, taxonomy strings, and DB hits:

In [39]:
df["sequence"].nunique()

42037

In [40]:
df["SILVA"].nunique()

453

In [41]:
df["dbhit"].nunique()

3184

So an alternative approach would be to group sequences by DB hit, and pick a random representative sequence.